In [4]:
from vosk import Model, KaldiRecognizer
import pyaudio
import json
import time
import os
import asyncio
import nest_asyncio
from dotenv import load_dotenv
from gtts import gTTS
import playsound
from langchain_google_genai import ChatGoogleGenerativeAI
# Initialize the Vosk model and recognizer
model = Model('vosk-model-small-en-in-0.4')
recognizer = KaldiRecognizer(model, 16000)

# Set up PyAudio for audio input
cap = pyaudio.PyAudio()

# Parameters for audio stream
channels = 1
rate = 16000
frames_per_buffer = 8192

# Start the audio stream
stream = cap.open(format=pyaudio.paInt16, channels=channels, rate=rate,
                  input=True, frames_per_buffer=frames_per_buffer)
print("Listening for your command...")

def llm_response(text):
    load_dotenv()
    google_gemini_api = os.getenv("GOOGLE_API_KEY")

    # Initialize the LLM model with API key
    llm_model = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=google_gemini_api)

    prompt = f"""
    you are anuradha a girl you have to answer like a girl make sure
    1. Clear and concise.
    2. Conversational and easy to understand.
    3. Suitable for text-to-speech conversion without any awkward phrasing or technical jargon.
    Here is the user's query: 
    {text}
    """

    # Predict the response from LLM
    llm_response = llm_model.predict(prompt)
    return llm_response


def genrate_voice(llm_output_for_voice):
    tts = gTTS(text=llm_output_for_voice, lang='en', slow=False,tld='co.in',lang_check=True)
    tts.save("temp.mp3")
    playsound.playsound("temp.mp3")
    os.remove("temp.mp3")
    
# Initialize variables
commands = []  # List to store completed commands
pause_duration = 2  # Pause duration in seconds to consider as finished
last_audio_time = time.time()  # Track when the last audio chunk was received

while True:
    # Read audio data from the stream
    data = stream.read(frames_per_buffer)

    if len(data) == 0:
        continue

    # Process the audio data with Vosk recognizer
    if recognizer.AcceptWaveform(data):
        result = recognizer.Result()
        result_dict = json.loads(result)
        text = result_dict.get('text', '')

        if text:
            # Append the recognized text directly to the commands list
            commands.append(text)
            
            # Show the latest transcription live (latest command)
            print(f"\rLive Transcription: {commands[-1]}", end="")

    # Check if there's been a pause of 2-3 seconds
    if time.time() - last_audio_time > pause_duration:
        # If there's a pause, finalize the transcription and print all commands so far
        if commands:  # Only process if there are any commands
            print("\nCompleted Commands:")
            for cmd in commands:
                print(f"- {cmd}")
            
            # Print the latest command (the most recent transcription)
            print(f"\nLatest Command: {commands[-1]}")
            llm_output=llm_response(commands[-1])
            genrate_voice(llm_output)

            # Reset for the next command
            print("Listening for the next command...")
        
        # Reset the last audio time to track when the next audio starts
        last_audio_time = time.time()

    # Update last audio time if new data is received
    last_audio_time = time.time()

# Stop the stream and close the PyAudio session
stream.stop_stream()
stream.close()
cap.terminate()



LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=10 max-active=3000 lattice-beam=2
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 1 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 2 orphan components.
LOG (VoskAPI:Collapse():nnet-utils.cc:1488) Added 1 components, removed 2
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from vosk-model-small-en-in-0.4/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from vosk-model-small-en-in-0.4/graph/HCLr.fst vosk-model-small-en-in-0.4/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:303) Loading winfo vosk-model-small-en-in-0.4/graph/phones/word_boundary.int


Listening for your command...
Live Transcription: what are you doing

KeyboardInterrupt: 

In [3]:
commands

['what are you doing today',
 'okay',
 'oh cavett night',
 'not listening very well']

In [ ]:
from gtts import gTTS
import playsound

# Your text to convert to speech
text = "Hello, this is a test of Google Text-to-Speech."

# Language in which you want to convert
language = 'en'  # English

# Passing the text and language to the engine, slow=False makes the speech faster
tts = gTTS(text=text, lang=language, slow=False,tld='co.in',lang_check=True)

# Saving the speech to a temporary file and playing it
tts.save("temp.mp3")
playsound.playsound("temp.mp3")

# Optionally, you can remove the temporary file after playing (use os.remove if needed)
import os
os.remove("temp.mp3")


In [ ]:
from vosk import Model, KaldiRecognizer
import pyaudio
import json
import time
import os
import asyncio
import nest_asyncio
from dotenv import load_dotenv
from gtts import gTTS
import playsound
from langchain_google_genai import ChatGoogleGenerativeAI

# Load environment variables
load_dotenv()

# Initialize the Vosk model and recognizer
model = Model('vosk-model-small-en-in-0.4')
recognizer = KaldiRecognizer(model, 16000)

# Set up PyAudio for audio input
cap = pyaudio.PyAudio()

# Parameters for audio stream
channels = 1
rate = 16000
frames_per_buffer = 8192

# Start the audio stream
stream = cap.open(format=pyaudio.paInt16, channels=channels, rate=rate,
                  input=True, frames_per_buffer=frames_per_buffer)
print("Listening for your command...")

# Function to get LLM response
def llm_response(text):
    google_gemini_api = os.getenv("GOOGLE_API_KEY")
    if not google_gemini_api:
        print("Error: GOOGLE_API_KEY is not set in environment variables.")
        return "Sorry, I couldn't get a response right now."
    
    # Initialize the LLM model with API key
    llm_model = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=google_gemini_api)

    prompt = f"""
    you are anuradha a girl you have to answer like a girl make sure
    1. Clear and concise.
    2. Conversational and easy to understand.
    3. Suitable for text-to-speech conversion without any awkward phrasing or technical jargon.
    Here is the user's query: 
    {text}
    """

    # Predict the response from LLM
    llm_response = llm_model.predict(prompt)
    return llm_response

# Function to generate voice from LLM output
def generate_voice(llm_output_for_voice):
    tts = gTTS(text=llm_output_for_voice, lang='en', slow=False, tld='co.in', lang_check=True)
    tts.save("temp.mp3")
    playsound.playsound("temp.mp3")
    os.remove("temp.mp3")

# Initialize variables
commands = []  # List to store completed commands
pause_duration = 2  # Pause duration in seconds to consider as finished
last_audio_time = time.time()  # Track when the last audio chunk was received

while True:
    # Read audio data from the stream
    data = stream.read(frames_per_buffer)

    if len(data) == 0:
        break

    # Process the audio data with Vosk recognizer
    if recognizer.AcceptWaveform(data):
        result = recognizer.Result()
        result_dict = json.loads(result)
        text = result_dict.get('text', '')

        if text:
            # Append the recognized text directly to the commands list
            commands.append(text)
            
            # Show the latest transcription live (latest command)
            print(f"\rLive Transcription: {commands[-1]}", end="")

    # Check if there's been a pause of 2-3 seconds
    if time.time() - last_audio_time > pause_duration:
        # If there's a pause, finalize the transcription and print all commands so far
        if commands:  # Only process if there are any commands
            print("\nCompleted Commands:")
            for cmd in commands:
                print(f"- {cmd}")
            
            # Print the latest command (the most recent transcription)
            print(f"\nLatest Command: {commands[-1]}")
            llm_output = llm_response(commands[-1])
            generate_voice(llm_output)

            # Reset for the next command
            print("Listening for the next command...")
        
        # Reset the last audio time to track when the next audio starts
        last_audio_time = time.time()

    # Update last_audio_time only when new data is received
    if len(data) > 0:
        last_audio_time = time.time()

# Stop the stream and close the PyAudio session
stream.stop_stream()
stream.close()
cap.terminate()


LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=10 max-active=3000 lattice-beam=2
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 1 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 2 orphan components.
LOG (VoskAPI:Collapse():nnet-utils.cc:1488) Added 1 components, removed 2
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from vosk-model-small-en-in-0.4/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from vosk-model-small-en-in-0.4/graph/HCLr.fst vosk-model-small-en-in-0.4/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:303) Loading winfo vosk-model-small-en-in-0.4/graph/phones/word_boundary.int


Listening for your command...
Live Transcription: okay exityou doing today

KeyboardInterrupt: 

In [9]:
from vosk import Model, KaldiRecognizer
import pyaudio
import json
import time
import os
import asyncio
import nest_asyncio
from dotenv import load_dotenv
from gtts import gTTS
import playsound
from langchain_google_genai import ChatGoogleGenerativeAI

# Load environment variables
load_dotenv()

# Initialize the Vosk model and recognizer
model = Model('vosk-model-small-en-in-0.4')
recognizer = KaldiRecognizer(model, 16000)

# Set up PyAudio for audio input
cap = pyaudio.PyAudio()

# Parameters for audio stream
channels = 1
rate = 16000
frames_per_buffer = 8192

# Start the audio stream
stream = cap.open(format=pyaudio.paInt16, channels=channels, rate=rate,
                  input=True, frames_per_buffer=frames_per_buffer)
print("Listening for your command...")

# Function to get LLM response
def llm_response(text):
    google_gemini_api = os.getenv("GOOGLE_API_KEY")
    if not google_gemini_api:
        print("Error: GOOGLE_API_KEY is not set in environment variables.")
        return "Sorry, I couldn't get a response right now."
    
    # Initialize the LLM model with API key
    llm_model = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=google_gemini_api)

    prompt = f"""
    you are anuradha a girl you have to answer like a girl make sure
    1. Clear and concise.
    2. Conversational and easy to understand.
    3. Suitable for text-to-speech conversion without any awkward phrasing or technical jargon.
    Here is the user's query: 
    {text}
    """

    # Predict the response from LLM
    llm_response = llm_model.predict(prompt)
    return llm_response

# Function to generate voice from LLM output
def generate_voice(llm_output_for_voice):
    tts = gTTS(text=llm_output_for_voice, lang='en', slow=False, tld='co.in', lang_check=True)
    tts.save("temp.mp3")
    playsound.playsound("temp.mp3")
    os.remove("temp.mp3")

# Initialize variables
commands = []  # List to store completed commands
pause_duration = 2  # Pause duration in seconds to consider as finished
last_audio_time = time.time()  # Track when the last audio chunk was received
command_received = False


while True:
    # Read audio data from the stream
    data = stream.read(frames_per_buffer)

    if len(data) == 0:
        continue

    # Process the audio data with Vosk recognizer
    if recognizer.AcceptWaveform(data):
        result = recognizer.Result()
        result_dict = json.loads(result)
        text = result_dict.get('text', '')

        if text:
            # Show the recognized text (command)
            commands.append(text)
            print(f"\rRecognized Command: {text}", end="")
            command_received = True  # Command has been recognized
            print(f"\nLatest Command: {commands[-1]}")
            llm_output = llm_response(commands[-1])
            generate_voice(llm_output)

            break  # Exit the loop after receiving one command

    # Check if there's been a pause of 2 seconds without audio
    if time.time() - last_audio_time > pause_duration:
        if command_received:
            # If a command was received, stop listening
            print("\nStopping listening after pause.")
            break  # Exit the loop after the pause
        
    # Update the last audio time if new data is received
    last_audio_time = time.time()

# Stop the stream and close the PyAudio session
stream.stop_stream()
stream.close()
cap.terminate()

LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=10 max-active=3000 lattice-beam=2
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 1 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 2 orphan components.
LOG (VoskAPI:Collapse():nnet-utils.cc:1488) Added 1 components, removed 2
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from vosk-model-small-en-in-0.4/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from vosk-model-small-en-in-0.4/graph/HCLr.fst vosk-model-small-en-in-0.4/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:303) Loading winfo vosk-model-small-en-in-0.4/graph/phones/word_boundary.int


Listening for your command...
Recognized Command: i wanted to know what is in apple
Latest Command: i wanted to know what is in apple


I0000 00:00:1732363340.494008  529347 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1732363340.494577  529347 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1732363340.909605  531193 subchannel.cc:806] subchannel 0x178b66080 {address=ipv6:%5B2404:6800:4009:811::200a%5D:443, args={grpc.client_channel_factory=0x10789dff0, grpc.default_authority=generativelanguage.googleapis.com:443, grpc.dns_enable_srv_queries=1, grpc.http2_scheme=https, grpc.internal.channel_credentials=0x17ae7eaf0, grpc.internal.client_channel_call_destination=0x1387bba18, grpc.internal.event_engine=0x178b62ff0, grpc.internal.security_connector=0x178b64d40, grpc.internal.subchannel_pool=0x14fffc590, grpc.max_receive_message_length=-1, grpc.max_send_message_length=-1, grpc.primary_user_agent=grpc-python/1.65.1, grpc.resource_quota=0x14fff8900, grpc.server_uri=dns:///generativelanguage.googleapis

In [14]:
import os
import time
import json
import asyncio
from vosk import Model, KaldiRecognizer
import pyaudio
from gtts import gTTS
import playsound
from langchain_google_genai import ChatGoogleGenerativeAI
from concurrent.futures import ThreadPoolExecutor

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Initialize the Vosk model and recognizer
model = Model('vosk-model-small-en-in-0.4')
recognizer = KaldiRecognizer(model, 16000)

# Set up PyAudio for audio input
cap = pyaudio.PyAudio()

# Parameters for audio stream
channels = 1
rate = 16000
frames_per_buffer = 4096  # Smaller buffer size for lower latency

# Start the audio stream
stream = cap.open(format=pyaudio.paInt16, channels=channels, rate=rate,
                  input=True, frames_per_buffer=frames_per_buffer)
print("Listening for your command...")

# Function to get LLM response asynchronously
async def llm_response(text):
    google_gemini_api = os.getenv("GOOGLE_API_KEY")
    if not google_gemini_api:
        print("Error: GOOGLE_API_KEY is not set in environment variables.")
        return "Sorry, I couldn't get a response right now."
    
    # Initialize the LLM model with API key
    llm_model = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=google_gemini_api)

    prompt = f"""
    you are anuradha a girl you have to answer like a girl make sure
    1. Clear and concise.
    2. Conversational and easy to understand.
    3. Suitable for text-to-speech conversion without any awkward phrasing or technical jargon.
    Here is the user's query: 
    {text}
    """

    # Predict the response from LLM
    llm_response = llm_model.predict(prompt)
    return llm_response

# Function to generate voice from LLM output
def generate_voice(llm_output_for_voice):
    tts = gTTS(text=llm_output_for_voice, lang='en', slow=False, tld='co.in', lang_check=True)
    tts.save("temp.mp3")
    playsound.playsound("temp.mp3")
    os.remove("temp.mp3")

# Initialize variables
commands = []  # List to store completed commands
pause_duration = 2  # Pause duration in seconds to consider as finished
command_received = False

# Initialize last_audio_time globally
last_audio_time = time.time()

# Create a thread pool to handle asynchronous tasks (LLM and TTS)
executor = ThreadPoolExecutor(max_workers=2)

# Run asynchronous tasks
async def main():
    global last_audio_time  # Ensure we reference the global variable
    while True:
        # Read audio data from the stream
        data = stream.read(frames_per_buffer)

        if len(data) == 0:
            continue

        # Process the audio data with Vosk recognizer
        if recognizer.AcceptWaveform(data):
            result = recognizer.Result()
            result_dict = json.loads(result)
            text = result_dict.get('text', '')

            if text:
                # Show the recognized text (command)
                commands.append(text)
                print(f"\rRecognized Command: {text}", end="")
                command_received = True  # Command has been recognized
                print(f"\nLatest Command: {commands[-1]}")

                # Generate response asynchronously
                llm_output = await llm_response(commands[-1])

                # Generate voice in the background using a thread
                executor.submit(generate_voice, llm_output)

                break  # Exit the loop after receiving one command

        # Check if there's been a pause of 2 seconds without audio
        if time.time() - last_audio_time > pause_duration:
            if command_received:
                # If a command was received, stop listening
                print("\nStopping listening after pause.")
                break  # Exit the loop after the pause

        # Update the last audio time if new data is received
        last_audio_time = time.time()

# Start the async main loop
asyncio.run(main())

# Stop the stream and close the PyAudio session
stream.stop_stream()
stream.close()
cap.terminate()


LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=10 max-active=3000 lattice-beam=2
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 1 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 2 orphan components.
LOG (VoskAPI:Collapse():nnet-utils.cc:1488) Added 1 components, removed 2
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from vosk-model-small-en-in-0.4/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from vosk-model-small-en-in-0.4/graph/HCLr.fst vosk-model-small-en-in-0.4/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:303) Loading winfo vosk-model-small-en-in-0.4/graph/phones/word_boundary.int


Listening for your command...
Recognized Command: tell me about her up a civilization
Latest Command: tell me about her up a civilization


I0000 00:00:1732363586.848966  529347 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1732363586.849452  529347 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1732363587.276728  531187 subchannel.cc:806] subchannel 0x168dddbb0 {address=ipv6:%5B2404:6800:4009:806::200a%5D:443, args={grpc.client_channel_factory=0x10789dff0, grpc.default_authority=generativelanguage.googleapis.com:443, grpc.dns_enable_srv_queries=1, grpc.http2_scheme=https, grpc.internal.channel_credentials=0x178ee7ac0, grpc.internal.client_channel_call_destination=0x1387bba18, grpc.internal.event_engine=0x168ddbcb0, grpc.internal.security_connector=0x168ddc870, grpc.internal.subchannel_pool=0x14fffc590, grpc.max_receive_message_length=-1, grpc.max_send_message_length=-1, grpc.primary_user_agent=grpc-python/1.65.1, grpc.resource_quota=0x14fff8900, grpc.server_uri=dns:///generativelanguage.googleapis

In [ ]:
import os
import time
import json
import logging
import asyncio
from vosk import Model, KaldiRecognizer
import pyaudio
from gtts import gTTS
import playsound
from langchain_google_genai import ChatGoogleGenerativeAI
from concurrent.futures import ThreadPoolExecutor
from dotenv import load_dotenv

# Set up logging configuration
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(message)s')
logger = logging.getLogger()

# Load environment variables
load_dotenv()

# Initialize the Vosk model and recognizer
model = Model('vosk-model-small-en-in-0.4')
recognizer = KaldiRecognizer(model, 16000)

# Set up PyAudio for audio input
cap = pyaudio.PyAudio()

# Parameters for audio stream
channels = 1
rate = 16000
frames_per_buffer = 4096  # Smaller buffer size for lower latency

# Start the audio stream
stream = cap.open(format=pyaudio.paInt16, channels=channels, rate=rate,
                  input=True, frames_per_buffer=frames_per_buffer)
logger.info("Listening for your command...")

# Function to get LLM response asynchronously
async def llm_response(text):
    google_gemini_api = os.getenv("GOOGLE_API_KEY")
    if not google_gemini_api:
        logger.error("GOOGLE_API_KEY is not set in environment variables.")
        return "Sorry, I couldn't get a response right now."
    
    # Initialize the LLM model with API key
    llm_model = ChatGoogleGenerativeAI(model="models/gemini-1.5-flash", google_api_key=google_gemini_api)

    prompt = f"""
    you are anuradha a girl you have to answer like a girl make sure
    1. Clear and concise.
    2. Conversational and easy to understand.
    3. Suitable for text-to-speech conversion without any awkward phrasing or technical jargon.
    Here is the user's query: 
    {text}
    """

    # Predict the response from LLM
    logger.info("Requesting LLM response for query: %s", text)
    llm_response = llm_model.predict(prompt)
    logger.info("Received LLM response.")
    return llm_response

# Function to generate voice from LLM output
def generate_voice(llm_output_for_voice):
    logger.info("Generating voice from LLM output...")
    tts = gTTS(text=llm_output_for_voice, lang='en', slow=False, tld='co.in', lang_check=True)
    tts.save("temp.mp3")
    playsound.playsound("temp.mp3")
    os.remove("temp.mp3")
    logger.info("Voice generated and played.")

# Initialize variables
commands = []  # List to store completed commands
pause_duration = 2  # Pause duration in seconds to consider as finished
command_received = False

# Initialize last_audio_time globally
last_audio_time = time.time()

# Create a thread pool to handle asynchronous tasks (LLM and TTS)
executor = ThreadPoolExecutor(max_workers=2)

# Run asynchronous tasks
async def main():
    global last_audio_time  # Ensure we reference the global variable
    while True:
        logger.debug("Reading audio data...")
        # Read audio data from the stream
        data = stream.read(frames_per_buffer)

        if len(data) == 0:
            continue

        # Process the audio data with Vosk recognizer
        logger.debug("Processing audio data with Vosk recognizer...")
        if recognizer.AcceptWaveform(data):
            result = recognizer.Result()
            result_dict = json.loads(result)
            text = result_dict.get('text', '')

            if text:
                # Show the recognized text (command)
                logger.info("Recognized Command: %s", text)
                commands.append(text)
                command_received = True  # Command has been recognized
                logger.debug(f"Latest Command: {commands[-1]}")

                # Generate response asynchronously
                logger.debug("Generating response from LLM...")
                llm_output = await llm_response(commands[-1])

                # Generate voice in the background using a thread
                executor.submit(generate_voice, llm_output)

                break  # Exit the loop after receiving one command

        # Check if there's been a pause of 2 seconds without audio
        if time.time() - last_audio_time > pause_duration:
            if command_received:
                # If a command was received, stop listening
                logger.info("Stopping listening after pause.")
                break  # Exit the loop after the pause

        # Update the last audio time if new data is received
        last_audio_time = time.time()

# Start the async main loop
logger.info("Starting main loop...")
asyncio.run(main())

# Stop the stream and close the PyAudio session
logger.info("Stopping the stream and closing PyAudio session.")
stream.stop_stream()
stream.close()
cap.terminate()


LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=10 max-active=3000 lattice-beam=2
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 1 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 2 orphan components.
LOG (VoskAPI:Collapse():nnet-utils.cc:1488) Added 1 components, removed 2
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from vosk-model-small-en-in-0.4/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from vosk-model-small-en-in-0.4/graph/HCLr.fst vosk-model-small-en-in-0.4/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:303) Loading winfo vosk-model-small-en-in-0.4/graph/phones/word_boundary.int
2024-11-23 17:42:58,076 - List

2024-11-23 17:43:21,043 - https://translate.google.co.in:443 "POST /_/TranslateWebserverUi/data/batchexecute HTTP/11" 200 None
2024-11-23 17:43:21,173 - headers-0: {'Referer': 'http://translate.google.com/', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36', 'Content-Type': 'application/x-www-form-urlencoded;charset=utf-8', 'Content-Length': '195'}
2024-11-23 17:43:21,175 - url-0: https://translate.google.co.in/_/TranslateWebserverUi/data/batchexecute
2024-11-23 17:43:21,175 - status-0: 200
2024-11-23 17:43:21,179 - part-0 written to <_io.BufferedWriter name='temp.mp3'>
2024-11-23 17:43:21,180 - part-0 created
2024-11-23 17:43:21,183 - Saved to temp.mp3
2024-11-23 17:43:25,335 - Voice generated and played.


In [9]:
import os
import time
import json
import logging
import pyaudio
from vosk import Model, KaldiRecognizer
import pyttsx3
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

# Set up logging configuration
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(message)s')
logger = logging.getLogger()

# Load environment variables
load_dotenv()

# Initialize the Vosk model and recognizer
model = Model('vosk-model-en-in-0.5')
recognizer = KaldiRecognizer(model, 16000)

# Set up PyAudio for audio input
cap = pyaudio.PyAudio()

# Parameters for audio stream
channels = 1
rate = 16000
frames_per_buffer = 4096  # Smaller buffer size for lower latency

# Start the audio stream
stream = cap.open(format=pyaudio.paInt16, channels=channels, rate=rate,
                  input=True, frames_per_buffer=frames_per_buffer)
logger.info("Listening for your command...")

# Function to get LLM response
def llm_response(text):
    google_gemini_api = os.getenv("GOOGLE_API_KEY")
    if not google_gemini_api:
        logger.error("GOOGLE_API_KEY is not set in environment variables.")
        return "Sorry, I couldn't get a response right now."
    
    llm_model = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=google_gemini_api)

    prompt = f"""
    You are Anuradha, a conversational assistant. Your responses should be:
    1. Clear and concise.
    2. Friendly and suitable for text-to-speech conversion.
    3. Free of technical jargon.
    Here is the user's query: 
    {text}
    """

    logger.info("Requesting LLM response for query: %s", text)
    return llm_model.predict(prompt)

# Function to generate voice from LLM output
def generate_voice(llm_output):
    logger.info("Generating voice from LLM output...")
    engine = pyttsx3.init()
    rate = engine.getProperty('rate')   # getting details of current speaking rate
    print (rate)                        #printing current voice rate
    engine.say(llm_output)
    engine.runAndWait()
    engine.stop()
    logger.info("Voice generation completed.")

# Main loop to listen and process a single command
try:
    while True:
        logger.debug("Reading audio data...")
        data = stream.read(frames_per_buffer)

        if len(data) == 0:
            continue

        # Process the audio data with Vosk recognizer
        if recognizer.AcceptWaveform(data):
            result = recognizer.Result()
            result_dict = json.loads(result)
            text = result_dict.get('text', '')

            if text:
                # Show the recognized text (command)
                logger.info("Recognized Command: %s", text)

                # Generate response from LLM
                llm_output = llm_response(text)

                # Generate and play voice response
                generate_voice(llm_output)

                # Exit after handling one command
                break
finally:
    # Stop the stream and close the PyAudio session
    logger.info("Stopping the stream and closing PyAudio session.")
    stream.stop_stream()
    stream.close()
    cap.terminate()
    import sys
    sys.exit()


LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=13 max-active=7000 lattice-beam=6
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 1 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 2 orphan components.
LOG (VoskAPI:Collapse():nnet-utils.cc:1488) Added 1 components, removed 2
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from vosk-model-en-in-0.5/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:279) Loading HCLG from vosk-model-en-in-0.5/graph/HCLG.fst
LOG (VoskAPI:ReadDataFiles():model.cc:294) Loading words from vosk-model-en-in-0.5/graph/words.txt
LOG (VoskAPI:ReadDataFiles():model.cc:303) Loading winfo vosk-model-en-in-0.5/graph/phones/word_boundar

200.0


2024-11-23 19:36:21,328 - Voice generation completed.
2024-11-23 19:36:21,359 - Stopping the stream and closing PyAudio session.


SystemExit: 

/Users/anuragsingh/Documents/GitHub/LLM/llmenv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
